<a href="https://colab.research.google.com/github/rafkaw/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'


/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])


    logdir = os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train, 
        y_train,
        batch_size=params_fit.get('batch_size',128),
        epochs=params_fit.get('epochs',5),
        verbose=params_fit.get('verbose',1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks=[tensorboard_callback]
        )
    return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
    return Sequential([
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),  
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same') 
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'),  
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'), 
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),                 
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'), 
      Conv2D(filters=128, kernel_size=(3, 3), activation='relu',padding='same'),  
      Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),                 
      MaxPool2D(),
      Dropout(0.3),


      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax'),
    ])

In [57]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 17s 64ms/step - loss: 2.2409 - accuracy: 0.3600 - val_loss: 0.5994 - val_accuracy: 0.8094
Epoch 2/5
272/272 [==============================] - 15s 54ms/step - loss: 0.5614 - accuracy: 0.8259 - val_loss: 0.1456 - val_accuracy: 0.9570
Epoch 3/5
272/272 [==============================] - 15s 54ms/step - loss: 0.2891 - accuracy: 0.9112 - val_loss: 0.0649 - val_accuracy: 0.9809
Epoch 4/5
272/272 [==============================] - 15s 54ms/step - loss: 0.1972 - accuracy: 0.9401 - val_loss: 0.0470 - val_accuracy: 0.9859
Epoch 5/5
272/272 [==============================] - 15s 54ms/step - loss: 0.1505 - accuracy: 0.9559 - val_loss: 0.0417 - val_accuracy: 0.9888


0.9551020408163265

In [45]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 1s 8ms/step - loss: 0.1711 - accuracy: 0.9653


0.9653061032295227

In [0]:
def get_model(params):
    return Sequential([
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),  
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'),
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu',padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'),  
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu',padding='same'),  
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),                 
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),  
      Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),                 
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),


      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax'),
    ])

In [0]:
def func_obj(params):
    model = get_model(params)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    model.fit(
        X_train, 
        y_train,
        batch_size=int(params.get('batch_size',128)),
        epochs=5,
        verbose=0,
    )
    
    score = model.evaluate(X_test, y_test, verbose=0)
    accuracy = score[1]
    print(params, 'accuracy{}'.format(accuracy))

    return {'loss':-accuracy, 'status': STATUS_OK, 'model': model}

In [60]:
space = {
    'bath_size': hp.quniform('bath_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.2, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'bath_size': 130.0, 'dropout_cnn_block_one': 0.42696768869943935, 'dropout_cnn_block_three': 0.3849951007510537, 'dropout_cnn_block_two': 0.43142192232338006, 'dropout_dense_block_one': 0.49192780862179253, 'dropout_dense_block_two': 0.6583727941615727}
accuracy0.940816342830658
{'bath_size': 160.0, 'dropout_cnn_block_one': 0.46067017505369234, 'dropout_cnn_block_three': 0.2064522988952247, 'dropout_cnn_block_two': 0.3016414992427962, 'dropout_dense_block_one': 0.36075586487862843, 'dropout_dense_block_two': 0.6339914092475877}
accuracy0.9575963616371155
{'bath_size': 130.0, 'dropout_cnn_block_one': 0.3371799766804914, 'dropout_cnn_block_three': 0.2817108046218536, 'dropout_cnn_block_two': 0.47331140911600067, 'dropout_dense_block_one': 0.3448321518237651, 'dropout_dense_block_two': 0.49327581523541986}
accuracy0.9700680375099182
{'bath_size': 200.0, 'dropout_cnn_block_one': 0.47811395640676146, 'dropout_cnn_block_three': 0.31506162722729464, 'dropout_cnn_block_two': 0.351046388211802